# Camera calibration & hand-eye coordination

## How to use
Put image source for calibration into 'images' directory.

In [1]:
import glob

import pandas as pd
import cv2
import numpy as np
import yaml
from sksurgerynditracker.nditracker import NDITracker

In [2]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255, 0, 0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0, 255, 0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0, 0, 255), 5)
    return img

In [3]:
class Calibrate:
    def __init__(self):
        print(
            """
==============================
  Camera calibration
------------------------------
Grid size
        """
        )
        self.grid_h = int(input("Grid height: "))
        self.grid_w = int(input("Grid width : "))

        self.point3d = np.zeros((self.grid_h * self.grid_w, 3), np.float32)
        self.point3d[:, :2] = np.mgrid[0 : self.grid_h, 0 : self.grid_w].T.reshape(
            -1, 2
        )

        point3d_list = []
        point2d_list = []
        self.criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        self.img_list = glob.glob("./images/img_*.jpg")
        assert len(self.img_list) != 0
        
        print('\nDetecting chess pattern...')

        for id_, file in enumerate(sorted(self.img_list)):
            frame = cv2.imread(file)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            ret, corners = cv2.findChessboardCorners(
                gray, (self.grid_h, self.grid_w), None
            )
            if ret:
                point3d_list.append(self.point3d)
                point2d_list.append(corners)
                corners_ = cv2.cornerSubPix(
                    gray, corners, (11, 11), (-1, -1), self.criteria
                )
                cv2.drawChessboardCorners(
                    frame, (self.grid_h, self.grid_w), corners_, True
                )
                cv2.imwrite("./images/point_{:05d}.jpg".format(id_ + 1), frame)
                cv2.waitKey(200)

        print("\nCalibrating...")
        ret, self.mtx, self.dist, rvecs, tvecs = cv2.calibrateCamera(
            point3d_list, point2d_list, gray.shape[::-1], None, None
        )
        pose_list = []
        for i in range(len(tvecs)):
            rot = cv2.Rodrigues(rvecs[i])[0].flatten()
            pose = np.hstack([tvecs[i].squeeze(), rot]).tolist()
            pose_list.append(pose)
        
        df = pd.DataFrame(pose_list, columns=[
                    "x",
                    "y",
                    "z",
                    "r1",
                    "r2",
                    "r3",
                    "r4",
                    "r5",
                    "r6",
                    "r7",
                    "r8",
                    "r9"])
        print(sorted(self.img_list))
        df.to_csv('extrinsic_param.csv', index=False)
        
        img_shape = frame.shape
        print(str(len(point2d_list)) + " frames are used.")
        print("Image size :", img_shape)
        print("RMS :", ret)
        print("Intrinsic parameters :")
        print(self.mtx)
        print("Distortion parameters :")
        print(self.dist)

        np.save("camera_config/intrinsic_paramter", self.mtx)
        np.save("camera_config/distortion_paramter", self.dist)

        with open("camera_config/camera_config.yaml", "w") as f:
            yaml.dump(
                {"image_size": {"h": img_shape[0], "w": img_shape[1]}, "rms": ret},
                f,
                default_flow_style=False,
            )

        print("Saved successfully !!\n")

        fs = cv2.FileStorage(
            "camera_config/calibration_result.xml", cv2.FILE_STORAGE_WRITE
        )
        fs.write("img_shape", img_shape)
        fs.write("rms", ret)
        fs.write("intrinsic", self.mtx)
        fs.write("distortion", self.dist)
        fs.release()
        cv2.destroyAllWindows()

    def test(self):
        print("\nStart testing")
        axis = np.float32([[1, 0, 0], [0, 1, 0], [0, 0, -1]]).reshape(-1, 3)

        for id_, file in enumerate(self.img_list):
            img = cv2.imread(file)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(
                gray, (self.grid_h, self.grid_w), None
            )

            if ret:
                corners_ = cv2.cornerSubPix(
                    gray, corners, (11, 11), (-1, -1), self.criteria
                )
                _, rvecs, tvecs, inliers = cv2.solvePnPRansac(
                    self.point3d, corners_, self.mtx, self.dist
                )
                imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, self.mtx, self.dist)
                img = draw(img, corners_, imgpts)
                cv2.imshow("img", img)
                cv2.waitKey(200)
                cv2.imwrite("./images/pose_{:05d}.jpg".format(id_ + 1), img)

        cv2.destroyAllWindows()

In [4]:
if __name__ == "__main__":
    calibrator = Calibrate()
    calibrator.test()


  Camera calibration
------------------------------
Grid size
        


Grid height:  7
Grid width :  5



Detecting chess pattern...

Calibrating...
['./images\\img_00018.jpg', './images\\img_00022.jpg', './images\\img_00025.jpg', './images\\img_00029.jpg', './images\\img_00040.jpg', './images\\img_00045.jpg', './images\\img_00054.jpg', './images\\img_00069.jpg', './images\\img_00074.jpg', './images\\img_00086.jpg', './images\\img_00092.jpg', './images\\img_00099.jpg', './images\\img_00101.jpg', './images\\img_00102.jpg', './images\\img_00113.jpg', './images\\img_00121.jpg', './images\\img_00131.jpg', './images\\img_00141.jpg']
18 frames are used.
Image size : (480, 720, 3)
RMS : 0.2102202378376349
Intrinsic parameters :
[[673.79197644   0.         314.61755151]
 [  0.         614.41734458 149.09453658]
 [  0.           0.           1.        ]]
Distortion parameters :
[[-3.61985793e-01  2.37435331e-01 -6.31252477e-03 -2.31660392e-04
  -1.11108879e-01]]
Saved successfully !!


Start testing
